# CHAPTER 7. 집계 연산
### 집계란
- 무언가를 함께 모으는 행위
- 집계를 수행하기 위해서는 **키나 그룹을 지정**하고 **하나 이상의 컬럼을 변환하는 방법을 지정하는 집계 함수를 사용**한다. 

### 그룹화 데이터 타입
- 가장 간단한 형태의 그룹화로 select 구문에서 집계를 수행한다.
- `group by`는 하나 이상의 키를 지정할 수 있으며 값을 가진 컬럼을 변환하기 위해 다른 집계 함수를 사용한다.
- `window`는 하나 이상의 키를 지정할 수 있으며, 값을 가진 컬럼을 변환하기 위해 다른 집계 함수를 사용할 수 있다. 단, 함수의 입력으로 사용할 로우는 현재 로우와 어느 정도 연관성이 있어야 한다.
- `grouping set`은 **서로 다른 레벨의 값을 집계할 때** 사용되며, SQL, DataFrame의 롤업 그리고 큐브를 사용할 수 있다.
- `roll up`은 하나 이상의 키를 지정할 수 있다. 컬럼을 변환하는 데 다른 집계 함수를 사용하여 **계층적으로 요약된 값을 구할 수 있다**.
- `cube`는 하나 이사의 키를 지정할 수 있으며 값을 가진 컬럼을 변환하기 위해 다른 집계 함수를 사용할 수 있다. 큐브는 모든 컬럼 조합에 대한 요약 값을 계산한다.

**지정된 집계 함수에 따라 그룹화된 결과는 RelationalGroupedDataset을 반환한다.

## 집계 함수
- 집계는 DataFrame의 .stat 속성을 이용하는 특별한 경우를 제외하곤 **함수를 사용**한다.
- org.apach.spark.sql.functions 패키지에서 참고

## (1) DataFrame 수준의 집계: 단일 컬럼 집계 함수

#### count
- count 함수는 액션이 아닌 트랜스포메이션으로 동작한다.
- 두 가지 방식으로 사용할 수 있다.
    - 1. count 함수에 특정 컬럼을 지정하는 방식
    - 2. count(*)나 count(1)을 사용하는 방식

#### countDistinct, approx_count_distinct
- countDistinct 함수는 고유 레코드 수를 구해야 할 때 사용된다. 따라서 **개별 컬럼을 처리하는 데 더 적합하다.**
- approx_count_distinct 함수는 최대 추정 오류율이라는 한 가지 파라미터를 더 사용한다.
- 예제에서는 0.1 이라는 큰 오류 설정으로 기대치에 크게 벗어나지만, countDistinct 함수보다 더 빠르게 결과를 반환한다. **따라서 이 함수의 성능은 대규모 데이터셋을 사용할 때 훨씬 더 좋아진다.**

In [1]:
# DataFrame을 사용해 기본 집계 수행, count 메서드를 사용한 예제
df = spark.range(500).toDF("number")
df.count() # count 메서드는 트렌스포메이션이 아닌 액션이므로 결과를 즉시 반환한다.

500

In [13]:
df = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("../../assets/exercises/week03/by-day/2010-12-01.csv")\
.coalesce(5)

df.cache()
df.createOrReplaceTempView('dfTable')
# 구매 이력 데이터를 사용해 파티션을 적은 수로 분할할 수 있도록 리파티셔닝하고
# 빠르게 접근할 수 있도록 캐싱한다.
# 여기서 파티션 수를 줄이는 이유는 적은 양의 데이터를 가진 수많은 파일이 존재하기 때문

In [8]:
df.columns

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country']

In [19]:
import pyspark.sql.functions as f

In [11]:
# from pyspark.sql.functions import count, countDistinct, approx_count_distinct

df.select(f.count('StockCode')).show() # 3108

+----------------+
|count(StockCode)|
+----------------+
|            3108|
+----------------+



In [9]:
df.select(f.countDistinct("StockCode")).show() # 1351

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     1351|
+-------------------------+



In [10]:
df.select(f.approx_count_distinct("StockCode", 0.1)).show() # 1382

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            1382|
+--------------------------------+



#### first와 last
- DataFrame의 첫 번째 값이나 마지막 값을 얻을 때 사용
- 단, DataFrame의 값이 아닌 로우를 기반으로 동작한다.

In [16]:
# from pyspark.sql.functions import first, last

df.select(f.first("StockCode"), f.last("StockCode")).show()

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|          85123A|          20755|
+----------------+---------------+



#### min과 max
- DataFrame에서 최솟값과 최댓값을 추출

In [18]:
# from pyspark.sql.functions import min, max
df.select(f.min("Quantity"), f.max("Quantity")).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|          -24|          600|
+-------------+-------------+



#### sum
- 특정 컬럼의 모든 값을 합산

#### sum_distinct(Deprecated : sumDistinct)
- 특정 컬럼의 고윳값을 합산

In [21]:
# from pyspark.sql.functions import sum, sumDistinct, sum_distinct

# df.select(f.sum("Quantity"), f.sumDistinct("Quantity")).show()
# FutureWarning: Deprecated in 3.2, use sum_distinct instead.
# warnings.warn("Deprecated in 3.2, use sum_distinct instead.", FutureWarning)

df.select(f.sum("Quantity"), f.sum_distinct("Quantity")).show()

+-------------+----------------------+
|sum(Quantity)|sum(DISTINCT Quantity)|
+-------------+----------------------+
|        26814|                  4690|
+-------------+----------------------+



#### avg
- 평균값
- sum 함수의 결과를 count 함수의 결과로 나누어 평균값을 구할 수 있지만 스파크의 avg 함수나 mean 함수를 사용하면 평균값을 더 쉽게 구할 수 있다.

In [22]:
# 이 예제는 집계된 컬럼을 재활용 하기 위해 alias 메소드를 사용한다.

df.select(
    f.count("Quantity").alias("total_transaction"),
    f.sum("Quantity").alias("total_purchase"),
    f.avg("Quantity").alias("avg_purchase"),
    f.expr("mean(Quantity)").alias("mean_purchase"),
).selectExpr(
    "total_purchase/total_transaction",
    "avg_purchase",
    "mean_purchase").show()


+------------------------------------+-----------------+-----------------+
|(total_purchase / total_transaction)|     avg_purchase|    mean_purchase|
+------------------------------------+-----------------+-----------------+
|                   8.627413127413128|8.627413127413128|8.627413127413128|
+------------------------------------+-----------------+-----------------+



#### 분산과 표준편차
- 분산은 평균과의 차이를 제곱한 결과의 평균이며 표준편차는 분산의 제곱근.
- 스파크에서는 표본표준편차 뿐만 아니라 모표준편차 방식도 지원하기 때문에 주의가 필요하다.
- variance와 stddev 함수를 사용하면 표본표준분산과 표본표준편차 공식을 이용
- 모표준분산이나 모표준편차 방식을 사용하려면 var_pop, stddev_pop 함수를 사용한다.

In [23]:
df.select(f.var_pop("Quantity"), f.var_samp("Quantity"),
         f.stddev_pop("Quantity"), f.stddev_samp("Quantity")).show()

+-----------------+------------------+--------------------+---------------------+
|var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+-----------------+------------------+--------------------+---------------------+
|695.2492099104054| 695.4729785650273|  26.367578764657278|   26.371821677029203|
+-----------------+------------------+--------------------+---------------------+



#### 비대칭과 첨도
- 데이터의 변곡점을 측정하는 방법
- 비대칭도: 데이터 평균의 비대칭 정도를 측정
- 첨도: 데이터 끝 부분을 측정

In [24]:
df.select(f.skewness("Quantity"), f.kurtosis("Quantity")).show()

+------------------+------------------+
|skewness(Quantity)|kurtosis(Quantity)|
+------------------+------------------+
|11.384721296581182|182.91886804842397|
+------------------+------------------+



## (2) DataFrame 수준의 집계:  두 컬럼값 사이의 영향도 비교

#### 공분산과 상관관계
- 공분산, cov
- 상관관계, corr : 피어슨 상관계수를 측정한다.

In [25]:
df.select(f.corr("InvoiceNo", "Quantity"), f.covar_samp("InvoiceNo", "Quantity"),
    f.covar_pop("InvoiceNo", "Quantity")).show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     -0.12225395743668731|            -235.56327681311157|            -235.4868448608685|
+-------------------------+-------------------------------+------------------------------+



## (3) DataFrame 수준의 집계:  복합 데이터 타입의 집계
특정 컬럼의 값을 리스트로 수집하거나 set() 데이터 타입으로 고윳값만 수집할 수 있다. 
수집된 데이터는 처리 파이프라인에서 다양한 프로그래밍 방식으로 다루거나 사용자 정의 함수를 사용해 전체 데이터에 접근할 수 있다.

In [26]:
df.agg(f.collect_set("Country"), f.collect_list("Country")).show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[France, Australi...| [United Kingdom, ...|
+--------------------+---------------------+



## (4) 데이터 그룹 기반 집계: 그룹화
**단일 컬럼의 데이터를 그룹화**하고 **해당 그룹의 다른 여러 컬럼을 사용해서 계산**하기 위해 카테고리형 데이터를 사용한다.

`고유한 송장번호(InvoiceNo)를 기준으로 그룹을 만들고 그룹별 물품 수를 카운트하기 위해서는` 집계 연산을 수행하는 두 단계로 이루어짐
- 1. 하나 이상의 컬럼을 그룹화한다.
    - RelationalGroupedDataset이 반환된다.
- 2. 그룹화 후에 집계 연산을 수행한다.
    - DataFrame이 반환된다.

In [28]:
df.groupby("InvoiceNo", "CustomerId").count().show()

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536596|      null|    6|
|   536530|   17905.0|   23|
|   536414|      null|    1|
|   536400|   13448.0|    1|
|   536550|      null|    1|
|   536522|   15012.0|   54|
|   536401|   15862.0|   64|
|   536409|   17908.0|   58|
|   536404|   16218.0|   28|
|   536562|   13468.0|   18|
|   536396|   17850.0|   18|
|   536539|   15165.0|   27|
|   536406|   17850.0|   17|
|   536523|   12868.0|   12|
|   536551|   17346.0|   35|
|   536567|   16048.0|    7|
|   536592|      null|  592|
|   536560|   13093.0|   13|
|   536593|   16835.0|    5|
|   536488|   17897.0|   35|
+---------+----------+-----+
only showing top 20 rows



#### 표현식을 이용한 그룹화
- 카운팅은 메서드로 사용할 수 있으므로 특별하지만, **메서드 대신 count 함수를 사용할 것을 권장**
- count 함수를 select 구문에 표현식으로 지정하는 것보다 agg 메서드를 사용하는 것이 좋다.
- agg 메서드는 여러 집계 처리를 한 번에 지정할 수 있으며, 집계에 표현식을 사용할 수 있다.
- 트랜스포메이션이 완료된 컬럼에 alias 메서드를 사용할 수 있다.

In [31]:
df.groupBy("InvoiceNo").agg(
    f.count("Quantity").alias("quan"),
    f.expr("count(Quantity)")
).show()

+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   536596|   6|              6|
|   536597|  28|             28|
|   536414|   1|              1|
|   536550|   1|              1|
|   536460|  14|             14|
|   536398|  17|             17|
|   536523|  12|             12|
|   536374|   1|              1|
|   536386|   3|              3|
|   536577|   4|              4|
|   536477|  14|             14|
|   536583|   1|              1|
|   536528|  57|             57|
|   536585|   1|              1|
|   536366|   2|              2|
|   536592| 592|            592|
|   536541|   1|              1|
|   536387|   5|              5|
|   536385|   7|              7|
|   536375|  16|             16|
+---------+----+---------------+
only showing top 20 rows



#### 맵을 이용한 그룹화
- 컬럼을 키로, 수행할 집계 함수의 문자열을 값으로 하는 맵 타입을 사용해 트랜스포메이션을 정의할 수 있다. 수행할 집계 함수를 한 줄로 작성하면 여러 컬럼명을 재사용할 수 있다.

In [36]:
df.groupBy("InvoiceNo").agg(f.expr("avg(Quantity)"), f.expr("stddev_pop(Quantity)")).show()

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536596|               1.5|  1.1180339887498947|
|   536597|2.5357142857142856|  2.7448932175059566|
|   536414|              56.0|                 0.0|
|   536550|               1.0|                 0.0|
|   536460|11.285714285714286|    8.80282885885937|
|   536398| 8.823529411764707|  4.9850989029972474|
|   536523| 9.333333333333334|   7.487025815072067|
|   536374|              32.0|                 0.0|
|   536386| 78.66666666666667|  30.169889330626027|
|   536577|              97.0|   47.52893855326458|
|   536477| 76.42857142857143|  121.20390981772016|
|   536583|             120.0|                 0.0|
|   536528| 2.175438596491228|   2.240949102012037|
|   536585|               2.0|                 0.0|
|   536366|               6.0|                 0.0|
|   536592|2.4966216216216215|    3.78430106261453|
|   536541| 

## (5) 윈도우 함수:
- 윈도우 함수는 데이터의 특정 '윈도우'를 대상을 고유한 집계 연산을 수행한다. 
- 데이터 윈도우는 현재 데이터에 대한 참조를 사용해 정의한다.
- 윈도우 명세는 함수에 전달될 로우를 결정한다.

#### 표준 group-by 함수와 차이점
- group-by 함수는 모든 로우 레코드가 단일 그룹으로만 이동한다.
- 윈도우 함수는 프레임에 입력되는 모든 로우에 대한 결괏값을 계산한다.
    - 프레임: 로우 그룹 기반의 테이블을 의미
        - 각 로우는 하나 이상의 프레임에 할당될 수 있다.
        - 롤링 평균을 구하기 위해서는 개별 로우가 7개의 다른 프레임으로 구성되어야 한다.
        
![윈도우 함수 시각화](../../assets/presentations/week07/spark7-visualizing_window_functions.png)
      
** 로우가 어떻게 여러 프레임에 할당될 수 있는지 나타낸다.


#### 랭크 함수(ranking function)
- dense_rank
    - 모든 고객에 대해 최대 구매 수량을 가진 날짜가 언제인지 확인한다.
    - 동일한 값이 나오거나 중복 로우가 발생해 순위가 비어 있을 수 있으므로 danse_rank 함수를 사용한다.

#### 분석 함수(analytic function)
#### 집계 함수(aggregate function)

#### 윈도우 함수 사용하기
1. 윈도우 함수를 정의하기 위해 **윈도우 명세**를 만든다.
- partitionBy 메서드는 지금까지 사용해온 파티셔닝 스키마의 개념과는 관련 없으며, 그룹을 어떻게 나눌지에 유사한 개념이다. OrderBy 메서드는 파티션의 정렬 방식을 정의한다.
- 프레임 명세 (rowsBetween 구문)는 입력된 로우의 참조를 기반으로 프레임에 로우가 포함될 수 있는지 결정한다.
- 예제에서는 첫 로우부터 현재 로우까지 확인한다.

2. 윈도우 명세를 이용해 확인하고 싶은 데이터를 위한 집계 함수 혹은 윈도우 함수를 준비한다.

3. select 구문에서 사용할 수 있는 컬럼을 반환한다. select 메서드를 사용해 계산된 윈도우값을 확인한다.

In [40]:
dfWithDate = df.withColumn("date", f.to_date(f.col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate.cache()
dfWithDate.createOrReplaceTempView("dfWithDate")

In [41]:
list(dfWithDate.columns)

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country',
 'date']

In [42]:
from pyspark.sql.window import Window

In [43]:
windowSpec = Window\
.partitionBy("CustomerId", "date")\
.orderBy(f.desc("Quantity"))\
.rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [45]:
# 시간대별 최대 구매를 구한다.
maxPurchaseQuantity = f.max(f.col("Quantity")).over(windowSpec)

In [46]:
# 구매량 순위
# 모든 고객에 대해 최대 구매 수량을 가진 날짜가 언제인지를 확인한다.
purchaseDenseRank = f.dense_rank().over(windowSpec)
purchaseRank = f.rank().over(windowSpec)

In [47]:
dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId")\
.select(
f.col("CustomerId"),
f.col("date"),
f.col("Quantity"),
purchaseRank.alias("purchaseRank"),
purchaseDenseRank.alias("purchaseDenseRank"),
maxPurchaseQuantity.alias("maxPurchaseQuantity")).show()

+----------+----------+--------+------------+-----------------+-------------------+
|CustomerId|      date|Quantity|purchaseRank|purchaseDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
|   12431.0|2010-12-01|      24|           1|                1|                 24|
|   12431.0|2010-12-01|      24|           1|                1|                 24|
|   12431.0|2010-12-01|      12|           3|                2|                 24|
|   12431.0|2010-12-01|       8|           4|                3|                 24|
|   12431.0|2010-12-01|       6|           5|                4|                 24|
|   12431.0|2010-12-01|       6|           5|                4|                 24|
|   12431.0|2010-12-01|       6|           5|                4|                 24|
|   12431.0|2010-12-01|       4|           8|                5|                 24|
|   12431.0|2010-12-01|       4|           8|                5|             

In [52]:
spark.sql("""SELECT CustomerId, date, Quantity,
    rank(Quantity) OVER (PARTITION BY CustomerId, date
                        ORDER BY Quantity DESC NULLS LAST
                        ROWS BETWEEN
                            UNBOUNDED PRECEDING AND
                            CURRENT ROW) as rank,
    dense_rank(Quantity) OVER (PARTITION BY CustomerId, date
                              ORDER BY Quantity DESC NULLS LAST
                              ROWS BETWEEN
                                  UNBOUNDED PRECEDING AND
                                  CURRENT ROW) as dRank,
    max(Quantity) OVER (PARTITION BY CustomerId, date
                       ORDER BY Quantity DESC NULLS LAST
                       ROWS BETWEEN
                           UNBOUNDED PRECEDING AND
                                  CURRENT ROW) as maxPurchase
FROM dfWithDate WHERE CustomerId IS NOT NULL ORDER BY CustomerId;""").show()

+----------+----------+--------+----+-----+-----------+
|CustomerId|      date|Quantity|rank|dRank|maxPurchase|
+----------+----------+--------+----+-----+-----------+
|   12431.0|2010-12-01|      24|   1|    1|         24|
|   12431.0|2010-12-01|       6|   5|    4|         24|
|   12431.0|2010-12-01|      12|   3|    2|         24|
|   12431.0|2010-12-01|      24|   1|    1|         24|
|   12431.0|2010-12-01|       3|  11|    6|         24|
|   12431.0|2010-12-01|       2|  12|    7|         24|
|   12431.0|2010-12-01|       2|  12|    7|         24|
|   12431.0|2010-12-01|       2|  12|    7|         24|
|   12431.0|2010-12-01|       6|   5|    4|         24|
|   12431.0|2010-12-01|       4|   8|    5|         24|
|   12431.0|2010-12-01|       8|   4|    3|         24|
|   12431.0|2010-12-01|       4|   8|    5|         24|
|   12431.0|2010-12-01|       4|   8|    5|         24|
|   12431.0|2010-12-01|       6|   5|    4|         24|
|   12433.0|2010-12-01|      96|   1|    1|     

## (6) 여러 그룹에 걸친 집계: 그룹화 셋(명시적인 그룹화)
- 여러 집계를 결합하는 저수준 기능이다.
- 그룹화 셋을 이용하면 group-by 구문에서 원하는 형태로 집계를 생성할 수 있다.
- 그룹화 셋은 null 값에 따라 집계 수준이 달라진다. null 값을 제거하지 않는다면 부정확한 결괏값을 얻게 된다.
- SQL 에서만 사용할 수 있다. DataFrame에서 동일한 연산을 수행하려면 rollup 메서드와 cube 메서드를 사용해야 한다.

#### group-by VS grouping sets
- stockCode와 CustomerId 별 총 수량은 group-by, grouping sets 모두 동일 결과를 얻을 수 있다.
- stockCode와 CustomerId에 상관없이 총 수량의 합산 결과를 추가하려면 group-by 구문을 사용해 처리하는 것은 불가능하다.


In [55]:
dfNoNull = dfWithDate.na.drop()
dfNoNull.createOrReplaceTempView("dfNoNull")

In [56]:
# group by 이용
spark.sql("""
SELECT CustomerId, stockCode, sum(Quantity)
FROM dfNoNull
GROUP BY customerId, stockCode
ORDER BY CustomerId DESC, stockCode DESC
""").show()

+----------+---------+-------------+
|CustomerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|   18229.0|    22848|            8|
|   18229.0|    22846|            8|
|   18229.0|    22730|            4|
|   18229.0|    22729|            4|
|   18229.0|    22728|            8|
|   18229.0|    22726|            8|
|   18229.0|    22725|            4|
|   18144.0|    84879|           80|
|   18144.0|    72817|           12|
|   18144.0|    21485|            3|
|   18085.0|   84029E|            8|
|   18085.0|    22837|            8|
|   18085.0|    22632|           12|
|   18085.0|    22619|            4|
|   18085.0|    22113|            8|
|   18085.0|    22111|           16|
|   18085.0|    21485|            8|
|   18085.0|    21481|            6|
|   18085.0|    21479|            8|
|   18074.0|    84755|           48|
+----------+---------+-------------+
only showing top 20 rows



In [58]:
# 그룹화 셋을 사용해 동일한 작업 수행
spark.sql("""
SELECT CustomerId, stockCode, sum(Quantity)
FROM dfNoNull
GROUP BY customerId, stockCode
GROUPING SETS ((customerId, stockCode))
ORDER BY CustomerId DESC, stockCode DESC
""").show()

+----------+---------+-------------+
|customerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|   18229.0|    22848|            8|
|   18229.0|    22846|            8|
|   18229.0|    22730|            4|
|   18229.0|    22729|            4|
|   18229.0|    22728|            8|
|   18229.0|    22726|            8|
|   18229.0|    22725|            4|
|   18144.0|    84879|           80|
|   18144.0|    72817|           12|
|   18144.0|    21485|            3|
|   18085.0|   84029E|            8|
|   18085.0|    22837|            8|
|   18085.0|    22632|           12|
|   18085.0|    22619|            4|
|   18085.0|    22113|            8|
|   18085.0|    22111|           16|
|   18085.0|    21485|            8|
|   18085.0|    21481|            6|
|   18085.0|    21479|            8|
|   18074.0|    84755|           48|
+----------+---------+-------------+
only showing top 20 rows



In [60]:
# 고객이나 재고 코드에 상관없이 총 수량의 합산 결과를 추가하려하면
# group-by 구문을 사용해 처리하는 것은 불가능
spark.sql("""
SELECT CustomerId, stockCode, sum(Quantity)
FROM dfNoNull
GROUP BY customerId, stockCode
GROUPING SETS ((customerId, stockCode), ())
ORDER BY sum(Quantity) DESC, CustomerId DESC, stockCode DESC
""").show()

+----------+---------+-------------+
|customerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|      null|     null|        24032|
|   13694.0|    17021|          600|
|   13777.0|    21232|          504|
|   16210.0|    21137|          480|
|   13777.0|   84029E|          480|
|   16029.0|    22466|          432|
|   16029.0|    21731|          432|
|   13777.0|    22095|          324|
|   13777.0|   85099B|          300|
|   17511.0|    20668|          288|
|   13777.0|   85123A|          256|
|   13694.0|    21154|          200|
|   16029.0|    79321|          192|
|   16029.0|    22780|          192|
|   16029.0|    22779|          192|
|   13777.0|    84050|          168|
|   17511.0|   84970S|          144|
|   17511.0|    21786|          144|
|   17181.0|    21326|          144|
|   16210.0|    22595|          144|
+----------+---------+-------------+
only showing top 20 rows



## (7) 여러 그룹에 걸친 집계: 롤업 (다차원 집계 기능)
- 다양한 컬럼을 그룹화 키로 설정하면 그룹화 키로 설정된 조합뿐만 아니라 데이터셋에서 볼 수 있는 실제 조합을 모두 살펴볼 수 있다.
- 롤업은 group-by 스타일의 다양한 연산을 수행할 수 있는 다차원 집계 기능이다.

- null 값은 가진 로우에서 전체 날짜의 합계를 확인할 수 있다.
- 롤업된 두 개의 컬럼값이 모두 null인 로우는 두 컬럼에 속한 레코드의 전체 함께를 나타낸다.

In [62]:
rolledUpDF = dfNoNull.rollup("Date", "Country").agg(f.sum("Quantity"))\
.selectExpr("Date", "Country", "`sum(Quantity)` as totla_quantity")\
.orderBy("Date")
rolledUpDF.show()

+----------+--------------+--------------+
|      Date|       Country|totla_quantity|
+----------+--------------+--------------+
|      null|          null|         24032|
|2010-12-01|     Australia|           107|
|2010-12-01|        France|           449|
|2010-12-01|United Kingdom|         21167|
|2010-12-01|          EIRE|           243|
|2010-12-01|          null|         24032|
|2010-12-01|        Norway|          1852|
|2010-12-01|       Germany|           117|
|2010-12-01|   Netherlands|            97|
+----------+--------------+--------------+



In [63]:
rolledUpDF.where("Country IS NULL").show()

+----------+-------+--------------+
|      Date|Country|totla_quantity|
+----------+-------+--------------+
|      null|   null|         24032|
|2010-12-01|   null|         24032|
+----------+-------+--------------+



## (8)  여러 그룹에 걸친 집계: 큐브
- 롤업을 고차원적으로 사용할 수 있게 해준다.
- 큐브는 요소들을 계층적으로 다루는 대신 모든 차원에 대해서 동일한 작업을 수행한다.
- 전체 기간에 대해 날짜의 국가별 결과를 얻을 수 있다.
    - 다음과 같은 정보를 가진 테이블을 만들 수 있음
        - 전체 날짜와 모든 국가에 대한 합계
        - 모든 국가의 날짜별 합계
        - 날짜별 국가별 합계
        - 전체 날짜의 국가별 합계
- 큐브를 사용해 테이블에 있는 모든 정보를 빠르고 쉽게 조회할 수 있는 요약 정보 테이블을 만들 수 있다.

In [64]:
dfNoNull.cube("Date", "Country").agg(f.sum("Quantity"))\
.selectExpr("Date", "Country", "`sum(Quantity)` as totla_quantity")\
.orderBy("Date").show()

+----------+--------------+--------------+
|      Date|       Country|totla_quantity|
+----------+--------------+--------------+
|      null|        France|           449|
|      null|          null|         24032|
|      null|     Australia|           107|
|      null|       Germany|           117|
|      null|        Norway|          1852|
|      null|United Kingdom|         21167|
|      null|          EIRE|           243|
|      null|   Netherlands|            97|
|2010-12-01|     Australia|           107|
|2010-12-01|United Kingdom|         21167|
|2010-12-01|          null|         24032|
|2010-12-01|   Netherlands|            97|
|2010-12-01|        France|           449|
|2010-12-01|        Norway|          1852|
|2010-12-01|       Germany|           117|
|2010-12-01|          EIRE|           243|
+----------+--------------+--------------+



In [67]:
dfNoNull.cube("customerId", "stockCode").agg(grouping_id(), sum("Quantity"))\
.orderBy(f.col("grouping_id()").desc)\
.show()

NameError: name 'grouping_id' is not defined

## 피벗
- 피벗을 사용해 로우를 컬럼으로 변환한다.
- 피벗을 사용해 국가별로 집계 함수를 적용할 수 있느며 쿼리를 사용해 쉽게 결과를 확인할 수 있다.

In [68]:
pivoted = dfWithDate.groupBy("date").pivot("Country").sum()

In [72]:
list(pivoted.columns)

['date',
 'Australia_sum(Quantity)',
 'Australia_sum(UnitPrice)',
 'Australia_sum(CustomerID)',
 'EIRE_sum(Quantity)',
 'EIRE_sum(UnitPrice)',
 'EIRE_sum(CustomerID)',
 'France_sum(Quantity)',
 'France_sum(UnitPrice)',
 'France_sum(CustomerID)',
 'Germany_sum(Quantity)',
 'Germany_sum(UnitPrice)',
 'Germany_sum(CustomerID)',
 'Netherlands_sum(Quantity)',
 'Netherlands_sum(UnitPrice)',
 'Netherlands_sum(CustomerID)',
 'Norway_sum(Quantity)',
 'Norway_sum(UnitPrice)',
 'Norway_sum(CustomerID)',
 'United Kingdom_sum(Quantity)',
 'United Kingdom_sum(UnitPrice)',
 'United Kingdom_sum(CustomerID)']